Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/explain-model/azure-integration/remote-explanation/explain-model-on-amlcompute.png)

## Setup

In [10]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.69


## Initialize a Workspace

Connect to the workspace

In [2]:
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

ignite
ignite
eastus2
15ae9cb6-95c1-483d-a0e3-b1a1a3b06324


In [3]:
model = ws.models['attrition_python']

### Create an environment for Scikit Learn

In [4]:
from azureml.core import Environment

env = Environment.from_conda_specification(name = "ScikitEnvironment", file_path='scikit.yaml')

env.register(ws)

{
    "name": "ScikitEnvironment",
    "version": "4",
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "python": {
        "userManagedDependencies": false,
        "interpreterPath": "python",
        "condaDependenciesFile": null,
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "dependencies": [
                "python=3.6.2",
                {
                    "pip": [
                        "azureml-defaults",
                        "scikit-learn==0.20.3",
                        "numpy",
                        "pandas",
                        "sklearn-pandas",
                        "inference-schema[numpy-support]",
                        "joblib"
                    ]
                }
            ],
            "name": "azureml_680cc38ac27203ae288d78d93091b187"
        }
    },
    "docker": {
        "enabled": false,
        "baseImage": "mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04",
 

In [5]:
# if already created, just retrieve it
env = ws.environments['ScikitEnvironment']

### Create an Inference Config for our Model

In [6]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py", environment=env)

In [7]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.exceptions import WebserviceException
from azureml.core import Model

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
aci_service_name = 'attritionpython'

try:
    # if you want to get existing service below is the command
    # since aci name needs to be unique in subscription deleting existing aci if any
    # we use aci_service_name to create azure aci
    service = Webservice(ws, name=aci_service_name)
    if service:
        service.delete()
except WebserviceException as e:
    print()

service = Model.deploy(ws, aci_service_name, [model], inference_config, deployment_config)



In [8]:
service.wait_for_deployment(True)
print(service.state)

Running.................................
SucceededACI service creation operation finished, operation "Succeeded"
Healthy


In [9]:
# if there are any issues whatsoever, there first place to look is here
print(service.get_logs())

2019-10-30T18:36:41,979316081+00:00 - gunicorn/run 
2019-10-30T18:36:41,982406987+00:00 - rsyslog/run 
2019-10-30T18:36:41,993340106+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_680cc38ac27203ae288d78d93091b187/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_680cc38ac27203ae288d78d93091b187/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_680cc38ac27203ae288d78d93091b187/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_680cc38ac27203ae288d78d93091b187/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_680cc38ac27203ae288d78d93091b187/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2019-10-30T18:36:42,003628225+00:00 - iot-server/run 
EdgeHubC

In [11]:
import pandas as pd
df = pd.read_csv('../data/IBM-Employee-Attrition.csv')

In [12]:
service.run(df.iloc[0:5].to_json())

[0.771333013718157,
 0.014618621100150676,
 0.5987369679687893,
 0.12480742395465541,
 0.3059415135073193]

## Visualize
Load the visualization dashboard